In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ana-verse-2-0-p/train.parquet
/kaggle/input/ana-verse-2-0-p/sample_submission.parquet
/kaggle/input/ana-verse-2-0-p/test.parquet


In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.ensemble import RandomForestClassifier


In [54]:
train = pd.read_parquet("/kaggle/input/ana-verse-2-0-p/train.parquet")
test = pd.read_parquet("/kaggle/input/ana-verse-2-0-p/test.parquet")
sample = pd.read_parquet("/kaggle/input/ana-verse-2-0-p/sample_submission.parquet")

train.head()


,Date,X1,X2,X3,X4,X5,target
0,2020-12-16,1.518921,5.463154,1.0,2.718282,2.890372,0
1,2020-12-16,1.546509,5.458010,1.0,2.718282,2.833213,1
2,2020-12-16,1.645427,5.456560,1.0,7.389056,2.890372,1
3,2020-12-16,1.652022,5.458479,1.0,2.718282,2.890372,1
4,2020-12-16,1.695538,5.466709,1.0,2.718282,2.890372,0


In [55]:
TARGET = "target"   

X = train.drop(columns=[TARGET])
y = train[TARGET]
X_test = test.copy()
datetime_cols = X.select_dtypes(include=["datetime64"]).columns
print("Datetime columns:", datetime_cols)
X = X.drop(columns=datetime_cols)
X_test = X_test.drop(columns=datetime_cols)


Datetime columns: Index(['Date'], dtype='object')


In [56]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

preprocess = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])



In [57]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train = preprocess.fit_transform(X_train)
X_val = preprocess.transform(X_val)


In [ ]:
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train, y_train)


In [ ]:
val_preds = model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, val_preds))
print("F1 Score:", f1_score(y_val, val_preds, average="macro"))
print(classification_report(y_val, val_preds))
X_full = preprocess.fit_transform(X)
model.fit(X_full, y)
